In [91]:
import pinecone
import numpy as np
import json
from langchain.vectorstores import Pinecone
import os
from getpass import getpass
from langchain.llms import Replicate
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import ConversationChain as chain
from langchain.memory import ConversationBufferMemory
from sentence_transformers import SentenceTransformer
import re
import string
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory, 
                                                  ConversationSummaryMemory, 
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain, StuffDocumentsChain

In [93]:
# your api key 

REPLICATE_API_TOKEN = getpass()

········


In [94]:
print(REPLICATE_API_TOKEN)

r8_8rCGHxwxv6TI0mQE8LhkeyJG49W2OdW2ojTUi


In [95]:
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

In [96]:
# max_new_tokens changes the max tokens allowed to be output by llama
# put your llama2 model in model 
llm = Replicate(
    model="",
    model_kwargs={"max_new_tokens": 8888, "temperature": 0.01} 
)

# Query from Pinecone and Resume Processing

In [1]:
# enter your api key and index name from your pinecone database
index_name = 'langchain-retrieval-augmentation'
pinecone.init( api_key='', 
              environment='us-west1-gcp-free' ) 
index = pinecone.Index('')

NameError: name 'pinecone' is not defined

In [7]:
# enter a job description
model = SentenceTransformer('BAAI/bge-small-en')
job_desc = [""]
embedded_job = model.encode(job_desc, normalize_embeddings=True)

NameError: name 'SentenceTransformer' is not defined

In [99]:
emb_query = np.array(embedded_job).tolist()

In [3]:
# queries pinecone database based on your embedded job description
out = index.query(
    vector = q3,
    namespace='',
    include_metadata=True, 
    top_k = 4)
out

NameError: name 'index' is not defined

In [4]:
extracted_contents = re.findall(r"'raw_text'(.*?)'score'", str(out), re.DOTALL)
extracted_contents = [content.strip() for content in extracted_contents]
extracted_contents[2]

NameError: name 're' is not defined

# get names from each resume


In [102]:
names = re.findall(r"'name'(.*?)'raw_text'", str(out), re.DOTALL)
names = [content.strip() for content in names]

In [5]:
name_list = []
for i in names:
    name = re.sub(r'\\n'," ", str(i))
    name = re.sub(r'[^A-Za-z0-9 ]+', ' ', name)
    name = " ".join(name.split())
    name_list.append(name)

print(name_list)

NameError: name 'names' is not defined

In [104]:
num_resumes = len(name_list)
num_resumes

4

# get resumes 

In [105]:
resume_list = []
# all replace/re.sub for symbols should occur before re.sub(r'[^A-Za-z0-9 ]+', ' ', s)
for i in extracted_contents:
    s = str(i).replace("\'", " ")
    s = s.replace("\\\\t", " ")
    s = s.replace("\\t", " ")
    s = re.sub(r'\n'," ", s)
    s = re.sub(r'/n', ' ', s)
    s = re.sub(r'\\n', ' ', s)
    s = re.sub(r'[^A-Za-z0-9 ]+', ' ', s)
    s = s.replace("raw_text :", " ")
    s = s[2:]
    s = " ".join(s.split())
    s = re.sub(r'uf0b7', '', s)
    resume_list.append(" New Resume: " + s)


# Map Reduce and Prompt Engineering

In [106]:
from langchain.schema import Document

docs = [
    Document(
        page_content=s
    )
    for s in resume_list
    
]
len(docs)

4

In [113]:
text_split = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    # the smaller chunk size the longer it takes to run
    chunk_size = 300, chunk_overlap = 2)
docs = text_split.split_documents(docs)

In [ ]:
# Map
map_template = """[INST] <<SYS>> You are a helpful assistant.'<</SYS>> 
The following contains""" + str(num_resumes) + """ resumes from """ + str(name_list) + """. 
Each candidate's resume is delimited with the phrase 'New Resume:' followed by the candidate's name, {docs}. 
Use ONLY the resumes provided and based on this list of resumes, please identify the key skills, experience, and education of each resume
Helpful Answer: [/INST]"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [115]:
# Reduce
reduce_template = """[INST] <<SYS>> You are a helpful assistant<</SYS>> The following contains the key skills of each person's resume:
{doc_summaries}
Take these and put it into a ranking of most skilled, most experience, and best education, please include the names of each resume 
and why you ranked them where they are. Helpful Answer: [/INST]"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [116]:
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name= "doc_summaries"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into. (not 1 to 1 with llama2's 4096 limit)
    token_max=3300,
)

In [117]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)


In [118]:
mapreduce_output = map_reduce_chain.run(docs)

In [6]:
# 3 resumes
print(mapreduce_output)

NameError: name 'mapreduce_output' is not defined

# Everything Put Together

In [121]:
def getresumes(job_desc, num):
    model = SentenceTransformer('BAAI/bge-small-en')
    embedded_job_desc = model.encode(str(job_desc), normalize_embeddings=True)

    resumes = index.query(
                vector = np.array(embedded_job_desc).tolist(),
                namespace='',
                include_metadata=True,
                top_k = int(num))
    
    extracted_contents = re.findall(r"'raw_text'(.*?)'score'", str(resumes), re.DOTALL)
    extracted_contents = [content.strip() for content in extracted_contents]
    
    resume_list = []
    
    # all replace/re.sub for symbols that include letters such as \\\\t should occur before re.sub(r'[^A-Za-z0-9 ]+', ' ', s)
    for i in extracted_contents:
        s = str(i).replace("\'", " ")
        s = s.replace("\\\\t", " ")
        s = s.replace("\\t", " ")
        s = re.sub(r'\n'," ", s)
        s = re.sub(r'/n', ' ', s)
        s = re.sub(r'\\n', ' ', s)
        s = re.sub(r'[^A-Za-z0-9 ]+', ' ', s)
        s = s.replace("raw_text :", " ")
        s = s[2:]
        s = " ".join(s.split())
        s = re.sub(r'uf0b7', '', s)
        resume_list.append(" New Resume: " + s)
    
    # make resume_list into Document type
    docs = [Document(page_content=s) for s in resume_list]
    
    name_list = []
    
    names = re.findall(r"'name'(.*?)'raw_text'", str(resumes), re.DOTALL)
    names = [content.strip() for content in names]
    for i in names:
        name = re.sub(r'\\n'," ", str(i))
        name = re.sub(r'[^A-Za-z0-9 ]+', ' ', name)
        name = " ".join(name.split())
        name_list.append(name)
    return(docs, name_list)        
    

In [122]:
def mapreduce(docs, name_list, map_prompt, reduceprompt):
    
    text_split = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    # the smaller chunk size the longer it takes to run
    chunk_size = 100, chunk_overlap = 0)
    docs = text_split.split_documents(docs)
    # Map
    map_template = """[INST] <<SYS>> Your job is to extract key components from each resume provided <</SYS>>
    The following contains""" + str(len(name_list)) + """ resumes from """ + str(name_list) + """. 
    Each candidate's resume is delimited with the phrase 'New Resume:' followed by the candidate's name, {docs}. 
    Using ONLY the resumes provided and based on this list of resumes,""" + map_prompt + """    
    Helpful Answer: [/INST]"""
    
    map_prompt = PromptTemplate.from_template(map_template)
    map_chain = LLMChain(llm=llm, prompt=map_prompt)
    
    # Reduce
    reduce_template = """[INST] <<SYS>> Your job is to rank candidates using only the information provided for a certain job<</SYS>> The following contains the key skills of each person's resume:
    {doc_summaries}. Please """ + reduceprompt + """
    Helpful Answer: [/INST]"""
    reduce_prompt = PromptTemplate.from_template(reduce_template)
    
    # Run chain
    reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

    # Takes a list of documents, combines them into a single string, and passes this to an LLMChain
    combine_documents_chain = StuffDocumentsChain(
        llm_chain=reduce_chain, document_variable_name= "doc_summaries"
    )

    # Combines and iteravely reduces the mapped documents
    reduce_documents_chain = ReduceDocumentsChain(
        # This is final chain that is called.
        combine_documents_chain=combine_documents_chain,
        # If documents exceed context for `StuffDocumentsChain`
        collapse_documents_chain=combine_documents_chain,
        # The maximum number of tokens to group documents into. (not 1 to 1 with llama2's 4096 limit)
        token_max=3300,
    )
    
    # Combining documents by mapping a chain over them, then combining results
    map_reduce_chain = MapReduceDocumentsChain(
        # Map chain
        llm_chain=map_chain,
        # Reduce chain
        reduce_documents_chain=reduce_documents_chain,
        # The variable name in the llm_chain to put the documents in
        document_variable_name="docs",
        # Return the results of the map steps in the output
        return_intermediate_steps=False,
    )
    
    output = map_reduce_chain.run(docs)
    return output


In [123]:
chatbot_template = """[INST] <<SYS>>You are an assistant to a recruiter that finds the best candidates for jobs. 
        You are to ONLY use the information provided to you to formulate your responses.
        You are designed to provide information on candidates based on the information given and if asked, evaluate candidates and give them an overall (based on multiple criteria such as education, relevant experience, and other skills) ranking from best match to worst match. 
        While doing so you should be able to justify why a candidate is ranked where they are.<</SYS>> [/INST]

        {history}
        Human: {human_input}
        Assistant:"""

chatbot_prompt = PromptTemplate(input_variables=["history", "human_input"], template=chatbot_template)
llm_chain = LLMChain(llm = llm,
                    prompt = chatbot_prompt,
                    verbose=False,
                    memory=ConversationBufferWindowMemory(k=3),
                    llm_kwargs={"max_length": 4096})

In [8]:
print(docs)

NameError: name 'docs' is not defined

In [9]:
while True:
    jobdesc = input("Please enter a job description: \n")
    num = input("Please enter the number of resumes you'd like: \n ")
    
    docs, names = getresumes(jobdesc, int(num))

    mapp = input("How would you like to map these resumes? \n")
    reduce = input("How would you like to reduce (rank) these resumes? \n")
    
    mapped_docs = mapreduce(docs, names, mapp, reduce)
    print(mapped_docs)
    
    while True:
        inn = input("What would you like to know from these resumes? \n")
        print(llm_chain.predict(human_input="[INST] <<SYS>> use only the information provided here:" + str(mapped_docs) + " to formulate your response <</SYS>>" + inn + "[/INST]"))
            
        if inn.lower() == 'exit':
            break
            
    in5 = input("type exit to exit \n")

    if in5.lower() == 'exit':
            break
            


KeyboardInterrupt: Interrupted by user